In [0]:
%sql
DESCRIBE EXTENDED dadosenergeticos.paises;

col_name,data_type,comment
id_pais,int,"Identificador único do país, usado como chave primária."
Entity,string,Nome oficial do país ou território.
Latitude,double,Coordenada de latitude geográfica em graus decimais.
Longitude,double,Coordenada de longitude geográfica em graus decimais.
landArea,double,"Área territorial total do país, em quilômetros quadrados (km²)."
PopulationalDensity,double,"Densidade populacional, expressa em habitantes por km²."
id_continente,int,Identificador do continente ao qual o país pertence.
,,
# Detailed Table Information,,
Catalog,spark_catalog,


In [0]:
%sql
DESCRIBE EXTENDED dadosenergeticos.dados;

col_name,data_type,comment
Year,bigint,"Ano da medição, no formato AAAA."
access_electricity,double,Percentual da população com acesso à eletricidade.
access_clean_fuels_cooking,double,Percentual da população com acesso a combustíveis limpos para cozinhar.
renewable_generating_capacity_per_capita,double,Capacidade instalada de geração de eletricidade renovável per capita (W/pessoa).
financial_flows_developing_countries,double,Fluxos financeiros destinados a países em desenvolvimento (US$).
renewable_share_final_energy_consumption,double,Participação das energias renováveis no consumo final de energia (%).
electricity_fossil_fuels,double,Geração de eletricidade a partir de combustíveis fósseis (TWh).
electricity_nuclear,double,Geração de eletricidade por fontes nucleares (TWh).
electricity_renewables,double,Geração de eletricidade por fontes renováveis (TWh).
low_carbon_generation,double,Proporção da geração de eletricidade de baixa emissão de carbono (%).


In [0]:
%sql
DESCRIBE EXTENDED dadosenergeticos.continente;

col_name,data_type,comment
id_continente,bigint,Identificador único do continente (chave primária)
Continente,string,"Nome do continente (ex.: Africa, North America, South America, Asia, Europe, Oceania, Antartica)"
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,dadosenergeticos,
Table,continente,
Created Time,Wed Apr 09 22:22:42 UTC 2025,
Last Access,UNKNOWN,
Created By,Spark 3.3.2,


In [0]:
%python
dados_pd = spark.table("dadosenergeticos.dados").toPandas()
dados_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3649 entries, 0 to 3648
Data columns (total 17 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Year                                      3649 non-null   int64  
 1   access_electricity                        3639 non-null   float64
 2   access_clean_fuels_cooking                3480 non-null   float64
 3   renewable_generating_capacity_per_capita  2718 non-null   float64
 4   financial_flows_developing_countries      1560 non-null   float64
 5   renewable_share_final_energy_consumption  3455 non-null   float64
 6   electricity_fossil_fuels                  3628 non-null   float64
 7   electricity_nuclear                       3523 non-null   float64
 8   electricity_renewables                    3628 non-null   float64
 9   low_carbon_generation                     3607 non-null   float64
 10  Primary_energy_consumption_per_capit

In [0]:
%python
paises_pd = spark.table("dadosenergeticos.paises").toPandas()
paises_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_pais              176 non-null    int32  
 1   Entity               176 non-null    object 
 2   Latitude             175 non-null    float64
 3   Longitude            175 non-null    float64
 4   landArea             175 non-null    float64
 5   PopulationalDensity  170 non-null    float64
 6   id_continente        176 non-null    int32  
dtypes: float64(4), int32(2), object(1)
memory usage: 8.4+ KB


Quais países possuem o maior consumo de energia total?


In [0]:
%sql
SELECT 
    p.Entity AS pais,
    ROUND(AVG(d.Primary_energy_consumption_per_capita), 2) AS media_consumo_per_capita_5_anos
FROM 
    dadosenergeticos.dados d
JOIN 
    dadosenergeticos.paises p ON d.id_pais = p.id_pais
WHERE 
    d.Year BETWEEN 2015 AND 2020
    AND d.Primary_energy_consumption_per_capita IS NOT NULL
GROUP BY 
    p.Entity
ORDER BY 
    media_consumo_per_capita_5_anos DESC
LIMIT 5;


pais,media_consumo_per_capita_5_anos
Qatar,211679.29
Iceland,179074.73
Singapore,159826.6
Bahrain,148473.13
United Arab Emirates,134892.35


In [0]:
%sql
SELECT 
    p.Entity AS pais,
    SUM(d.Primary_energy_consumption_per_capita) AS total_consumo_per_capita_5_anos
FROM 
    dadosenergeticos.dados d
JOIN 
    dadosenergeticos.paises p ON d.id_pais = p.id_pais
WHERE 
    d.Year BETWEEN 2019 AND 2024
    AND d.Primary_energy_consumption_per_capita IS NOT NULL
GROUP BY 
    p.Entity
ORDER BY 
    total_consumo_per_capita_5_anos DESC
LIMIT 5;

pais,total_consumo_per_capita_5_anos
Qatar,392095.86
Iceland,331282.33
Singapore,312914.19
Bahrain,311494.27
United Arab Emirates,260952.05000000002


In [0]:
%sql
SELECT 
    p.Entity AS pais,
    ROUND(SUM(
        d.electricity_fossil_fuels +
        d.electricity_nuclear +
        d.electricity_renewables
    ), 2) AS total_eletricidade_5_anos,
    ROUND(AVG(d.Primary_energy_consumption_per_capita), 2) AS media_consumo_per_capita_5_anos
FROM 
    dadosenergeticos.dados d
JOIN 
    dadosenergeticos.paises p ON d.id_pais = p.id_pais
WHERE 
    d.Year BETWEEN 2015 AND 2020
    AND d.electricity_fossil_fuels IS NOT NULL
    AND d.electricity_nuclear IS NOT NULL
    AND d.electricity_renewables IS NOT NULL
    AND d.Primary_energy_consumption_per_capita IS NOT NULL
GROUP BY 
    p.Entity
ORDER BY 
    total_eletricidade_5_anos DESC
LIMIT 5;


pais,total_eletricidade_5_anos,media_consumo_per_capita_5_anos
China,40754.39,26813.78
United States,24626.97,78254.74
India,8958.35,6367.0
Japan,5987.22,40861.68
Canada,3816.42,108686.58


Quais países são mais dependentes de fontes não renováveis?

In [0]:
%sql
SELECT     
    p.Entity AS pais,
    d.Year,
    ROUND(
        100 * SUM(d.electricity_fossil_fuels + d.electricity_nuclear) / 
        SUM(d.electricity_fossil_fuels + d.electricity_nuclear + d.electricity_renewables), 
        2
    ) AS percentual_nao_renovavel
FROM     
    dadosenergeticos.dados d
JOIN     
    dadosenergeticos.paises p ON d.id_pais = p.id_pais
WHERE     
    d.Year = 2020

GROUP BY     
    p.Entity, d.Year
ORDER BY     
    percentual_nao_renovavel DESC;


pais,Year,percentual_nao_renovavel
Nauru,2020,100.0
Gambia,2020,100.0
Guinea-Bissau,2020,100.0
Grenada,2020,100.0
Djibouti,2020,100.0
Kiribati,2020,100.0
Turkmenistan,2020,100.0
Saint Lucia,2020,100.0
Solomon Islands,2020,100.0
Bahamas,2020,100.0


In [0]:
%sql
SELECT     
    p.Entity AS pais,
    d.Year,
    ROUND(
        100 * SUM(d.electricity_renewables) / 
        SUM(d.electricity_fossil_fuels + d.electricity_nuclear + d.electricity_renewables), 
        2
    ) AS percentual_renovavel
FROM     
    dadosenergeticos.dados d
JOIN     
    dadosenergeticos.paises p ON d.id_pais = p.id_pais
WHERE     
    d.Year = 2020

GROUP BY     
    p.Entity, d.Year
ORDER BY     
    percentual_renovavel DESC
LIMIT 10;


pais,Year,percentual_renovavel
Nepal,2020,100.0
Lesotho,2020,100.0
Iceland,2020,100.0
Central African Republic,2020,100.0
Albania,2020,100.0
Bhutan,2020,100.0
Ethiopia,2020,99.93
Paraguay,2020,99.85
Norway,2020,98.57
Costa Rica,2020,98.38


In [0]:
%sql
SELECT 
    c.Continente AS Continente,
    ROUND(SUM(d.electricity_fossil_fuels + d.electricity_nuclear + d.electricity_renewables), 2) AS total_eletricidade_5_anos,
    ROUND(AVG(d.Primary_energy_consumption_per_capita), 2) AS media_consumo_per_capita_5_anos
FROM 
    dadosenergeticos.dados d
JOIN 
    dadosenergeticos.paises p ON d.id_pais = p.id_pais
JOIN 
    dadosenergeticos.continente c ON p.id_continente = c.id_continente
WHERE 
    d.Year BETWEEN 2015 AND 2020
    AND d.electricity_fossil_fuels IS NOT NULL
    AND d.electricity_nuclear IS NOT NULL
    AND d.electricity_renewables IS NOT NULL
    AND d.Primary_energy_consumption_per_capita IS NOT NULL
GROUP BY 
    c.Continente
ORDER BY 
    total_eletricidade_5_anos DESC;


Continente,total_eletricidade_5_anos,media_consumo_per_capita_5_anos
Asia,63124.09,37368.73
North America,31116.38,28416.36
Europe,22091.49,42181.9
South America,5763.16,16078.87
Africa,4615.16,5564.52
Oceania,1791.09,22916.69


In [0]:
%sql
SELECT 
    c.Continente AS continente,
    ROUND(SUM(
        d.electricity_fossil_fuels +
        d.electricity_nuclear +
        d.electricity_renewables
    ), 2) AS total_eletricidade_5_anos_TWh,
    
    ROUND(AVG(d.Primary_energy_consumption_per_capita), 2) AS media_consumo_per_capita_kWh,
    
    ROUND(AVG(d.gdp_per_capita), 2) AS media_pib_per_capita_usd

FROM 
    dadosenergeticos.dados d
JOIN 
    dadosenergeticos.paises p ON d.id_pais = p.id_pais
JOIN 
    dadosenergeticos.continente c ON p.id_continente = c.id_continente
WHERE 
    d.Year BETWEEN 2015 AND 2020
    AND d.electricity_fossil_fuels IS NOT NULL
    AND d.electricity_nuclear IS NOT NULL
    AND d.electricity_renewables IS NOT NULL
    AND d.Primary_energy_consumption_per_capita IS NOT NULL
    AND d.gdp_per_capita IS NOT NULL
GROUP BY 
    c.Continente
ORDER BY 
    total_eletricidade_5_anos_TWh DESC;


continente,total_eletricidade_5_anos_TWh,media_consumo_per_capita_kWh,media_pib_per_capita_usd
Asia,62992.33,39167.18,13304.29
North America,31098.46,29679.88,21708.35
Europe,21424.62,42330.61,32672.92
South America,5763.16,16078.87,8320.56
Africa,3433.3,5796.76,2595.24
Oceania,1787.94,22126.76,14606.07
